In [1]:
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt


In [2]:
#helpful links
# example solution: https://www.kaggle.com/code/caesarmario/survival-prediction-w-various-ml-models/notebook#4.-|-Reading-Dataset-%F0%9F%91%93
# helpful figure: https://imgur.com/zbFvhx8
# how to bin: https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950
# see notebook for rough steps: imputing, binning, one-hot encoding, drop unused columns

In [3]:
# To-do
# - encode text features
# - deal with features with no data point
# - add all this to a pipeline and process the test and train dataset
# - train and evalute the model

In [4]:
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')
test_Ids = test["PassengerId"]
print(len(train))
print(len(test))


891
418


In [5]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
#define a function for cleaning data
def clean(data):
    # fill the Age, Fare, Parch and SibSp with average
    data['Age'].fillna(data['Age'].mean(), inplace = True)
    data['Fare'].fillna(data['Fare'].mean(), inplace = True)
    data['Parch'].fillna(data['Parch'].mean(), inplace = True)
    data['SibSp'].fillna(data['SibSp'].mean(), inplace = True)

    # fill empty "Embarked" with 'X'
    data['Embarked'].fillna("X", inplace = True)

    # drop passengerId, Ticket, Name
    data.drop(['PassengerId'], axis=1, inplace = True)
    data.drop(['Ticket'], axis=1, inplace = True)
    data.drop(['Name'], axis=1, inplace = True)
    data.drop(['Cabin'], axis=1, inplace = True)

    return data

In [7]:
train = clean(train)
test = clean(test)
train.head(5)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
# Encode string lables to integers
from sklearn import preprocessing
label_enc = preprocessing.LabelEncoder()
cols = ['Sex', 'Embarked']

def encode_labels(data):
    for col in cols:
        data[col] = label_enc.fit_transform(data[col])
        print(label_enc.classes_)


In [9]:
encode_labels(train)
encode_labels(test)

['female' 'male']
['C' 'Q' 'S' 'X']
['female' 'male']
['C' 'Q' 'S']


In [10]:
# split training data into train and validation sets
from sklearn.model_selection import train_test_split
y = train['Survived']
X = train.drop('Survived', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15)

In [11]:
# #kNN test
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=11)
# neigh.fit(X_train,y_train)

# pred = neigh.predict(X_val)

In [12]:
# #Decision tree test
# from sklearn.tree import DecisionTreeClassifier
# tree_clf = DecisionTreeClassifier(random_state=0)
# tree_clf.fit(X_train,y_train)

# pred = tree_clf.predict(X_val)

In [13]:
# #Naieve Bayes test
# from sklearn.naive_bayes import GaussianNB
# nb_clf = GaussianNB()
# nb_clf.fit(X_train,y_train)

# pred = nb_clf.predict(X_val)

In [14]:
from sklearn.linear_model import LogisticRegression
log_regr = LogisticRegression()
log_regr.fit(X_train,y_train)

pred = log_regr.predict(X_val)

In [15]:
# test selected model on validation set
from sklearn.metrics import accuracy_score
accuracy_score(y_val, pred)

0.8059701492537313

In [16]:
# Note: uncomment whichever classifier you used trained above
# Create submission data
# sub = neigh.predict(test)
# sub = tree_clf.predict(test)
# sub = nb_clf.predict(test)
sub = log_regr.predict(test)

In [17]:
df = pd.DataFrame({'PassengerId': test_Ids.values,'Survived': sub})
df.to_csv('submitMe.csv', index=False)